In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib_inline
%matplotlib inline

In [2]:
# for better viz
import pprint
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read player data
# This contains only players that started career in 2017 or earlier
player_data = pd.read_csv("Data/atp_players.csv")

# This contains all players up to date
player_data_additional = pd.read_csv("Data/tennis_atp-master/atp_players.csv")

# Main dataset
match_data = {}
whole_match_data = pd.DataFrame()

# Additional dataset
match_data_additional = {}
whole_match_data_additional = pd.DataFrame()

# Read match data in one DataFrame
for year in range(2000, 2022):
    data = pd.read_csv(f"Data/{year}.csv")
    match_data[str(year)] = data.copy()
    whole_match_data = pd.concat([whole_match_data, data])
    
    data = pd.read_csv(f"Data/tennis_atp-master/atp_matches_{year}.csv")
    match_data_additional[str(year)] = data.copy()
    whole_match_data_additional = pd.concat([whole_match_data_additional, data])
    
# Reset index to have proper values
whole_match_data = whole_match_data.reset_index()
whole_match_data_additional = whole_match_data_additional.reset_index()

## Select data

### Select data source
| # | data | included/excluded | reasons | quality | volume/data types |
|:---:|:---|:---|:---|:---|:---|
| 1 | atp_players.csv | excluded | Does not have information about players past 2017. All inforation can be extracted from other source. | - | - |
| 2 | (2000-2021).csv | included | Have necessary information about matches and odds. | Average. Have some nans and strange duplicates | ~58k, dtypes - int, float, string |
| 3 | tennis_atp-master/atp_players.csv | included | Have up-to-date information about players | Average. Have nans | ~55k, dtypes - int, string |
| 4 | tennis_atp-master/atp_matches_(2000-2021).csv | included | Have additional information about player, such as height and main hand. | Average. Have nans in values that possibly can be used, if quality would better | ~66k, dtypes - int, string, float |



### (2000-2021).csv
#### Select attributes
| # | attributes | included/excluded | reasons | quality | data type |
|:---:|:---|:---|:---|:---|:---|
| 1 | atp | Included | Used for formating date | Good | Int |
| 2 | location | Included | Can be used to determine home player | Good | String |
| 3 | tournament | Excluded | Possibly may be used for join tables, but tournament naming is different in different data sources | Good | String |
| 4 | date | Included | Can be used for prediction and join | Good | Datetime |
| 5 | series | Included | Can be used for prediction because represents the tournament level | Good | String |
| 6 | court | Included | Can be used for prediction | Good | String |
| 7 | surface | Included | Can be used for prediction | Good | String |
| 8 | round | Included  | Can be used for prediction and join | Good | String |
| 9 | best_of | Included | Can be used for prediction and join | Good | String |
| 10 | winner / loser | Included | Main information about match | Good | String |
| 11 | wrank / lrank | Included | Can be user for prediction and join | Average | Int |
| 12 | wpts / lpts | Excluded | Same info as wrank, but with lot of nans | Average | Int |
| 13 | w1-w5 / l1-l5 | Exluded | Future information, can't be used for prediction | Good | Int |
| 14 | wsets / lsets | Excluded | Future information, can't be used for prediction | Average | Int |
| 15 | comment | Included | Can be used to sort out uncompleted matches | Good | String |
| 16 | (*)w/l | Included | Betting odds from different companies. Can be used to fill nan values in maxw/maxl and avgw/avgl | Average | float |
| 17 | maxw / maxl | Included | Main information for evaluation. Represents max odds among all companies. | Bad - Have lot of nans | float |
| 18 | avgw / avgl | Included | Main information for evaluation. Represents average odds among all companies. | Bad - Have lot of nans | float |

#### Select records
| # | records | included/excluded | reasons |
|:---:|:---|:---|:---|
| 1 | Mathces with comment != 'Completed' | excluded | We interested only in completed matches |

### tennis_atp-master/atp_players.csv
#### Select attributes

| # | attributes | included/excluded | reasons | quality | data type |
|:---:|:---|:---|:---|:---|:---|
| 1 | player_id | Included | Need to replace id to names in matches datra | Good | Int |
| 2 | name_first / name_last | Included | Player's names. Need to track player performance and join the tables | Good | String |
| 3 | hand | Included | Can be used to fill nans | Average | String |
| 4 | dob | Excluded | Date of birth. Already have age info in matches data | Average | Int |
| 5 | ioc | Included | Flag code. Can be used to fill nans in height among different countries | Good | String |
| 6 | height | Included | Can be used to fill nans | Bad | Int |
| 7 | wikidata_id | Excluded | Possibly can be used for finding additional information, but already have all we needed | Bad | Int |

#### Select records
| # | records | included/excluded | reasons |
|:---:|:---|:---|:---|
| 1 | Players that are not presented in atp_matches | excluded | Not interested in players outside dataset |

### tennis_atp-master/atp_matches_(2000-2021).csv
#### Select attributes

| # | attributes | included/excluded | reasons | quality | data type |
|:---:|:---|:---|:---|:---|:---|
| 1 | tourney_id | Excluded | Can't be used for prediction and join | Good | String |
| 2 | tourney_name | Excluded | Possibly can be used for prediction, but tournament level fit's better | Good | String |
| 3 | surface | Included | Can be used for join previous match data | Good | String |
| 4 | draw_size | Included | Can be used for prediction and joining | Good | Int |
| 5 | tourney_level | Included | Represents tournament level. Can be used for prediction | Good | String |
| 6 | tourney_date | Included | Can be used for selecting and joining tables | Good | String |
| 7 | match_num | Excluded | Just serial number in year | Good | Int |
| 8 | winner_id / loser_id | Included | Info about players | Good | Int |
| 9 | winner_seed / loser_seed | Excluded | Don't need | Average | Int |
| 10 | winner_entry / loser_entry | Excluded | Possibly can be used for prediction, but 90% of nulls | Bad | String |
| 11 | winner_name / loser_name | Included | Main info about player | Good | String |
| 12 | winner_hand / loser_hand | Included | Can be used for prediction | Average | String |
| 13 | winner_ht / loser_ht | Included | Player's height. Can be used for prediction | Average | Int |
| 14 | winner_ioc / loser_ioc | Included | Player's flag code. Can be used for prediction | Good | String |
| 15 | winner_age / loser_age | Included | Player's age. Can be used for prediction | Good | Float |
| 16 | score | Exluded | Future info. | Good | String |
| 17 | best_of | Included | Can be used for prediction and join | Good | Int |
| 18 | round | Included | Round of tournament. Can be used for prediction and joining. | Good | Int |
| 19 | minutes | Excluded | Future info. | Average | Float |
| 20 | (w/l)_ace | Excluded | Future info. | Average | Float |
| 21 | (w/l)_df | Excluded | Future info. | Average | Float |
| 22 | (w/l)_svpt | Excluded | Future info. | Average | Float |
| 23 | (w/l)_1stIn | Excluded | Future info. | Average | Float |
| 24 | (w/l)_1stWon | Excluded | Future info. | Average | Float |
| 25 | (w/l)_2ndWon | Excluded | Future info. | Average | Float |
| 26 | (w/l)_SvGms | Excluded | Future info. | Average | Float |
| 27 | (w/l)_bpSaved | Excluded | Future info. | Average | Float |
| 28 | (w/l)_bpFaced | Excluded | Future info. | Average | Float |
| 29 | winner_rank / loser_rank | Included | Rank before the match. Can be used for predciton and joining. | Average | Float |
| 30 | winner_rank_points / loser_rank_points | Excluded | Rank points before the match. Same info as winner rank, but more nans | Average | Float |

## Clean data

### Drop unimportant info
#### (2000-2021).csv

In [4]:
# Drop columns
whole_match_data = whole_match_data.drop(
    ["tournament", "w1", "w2", "w3", "w4", "w5",
     "l1", "l2", "l3", "l4", "l5", "wsets", "lsets", "wpts", "lpts"], axis=1)

# Drop non-completed
whole_match_data = whole_match_data.drop(whole_match_data[whole_match_data["comment"] != "Completed"].index)

# Drop comment column
whole_match_data = whole_match_data.drop(["comment"], axis=1)

In [5]:
whole_match_data.head(3)

,index,atp,location,date,series,court,surface,round,best_of,winner,...,ubw,ubl,lbw,lbl,sjw,sjl,maxw,maxl,avgw,avgl
0,0,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Player data additional

In [6]:
player_data_additional = player_data_additional.drop(["dob", "wikidata_id"], axis=1)

In [7]:
# Get all players from matches dataset
players = set()

for player in whole_match_data_additional.iloc:
    players |= set([player["winner_id"]]) | set([player["loser_id"]])

In [8]:
players = list(players)
len(players)

2399

In [9]:
# Filter players
player_data_additional = player_data_additional[player_data_additional["player_id"].isin(players)]
player_data_additional = player_data_additional.reset_index()
player_data_additional = player_data_additional.drop("index", axis=1)
player_data_additional

,player_id,name_first,name_last,hand,ioc,height
0,100644,Alexander,Zverev,R,GER,198.0
1,100754,Rafael,Avalos Brenes,R,CRC,NaN
2,100943,Girts,Dzelde,R,LAT,173.0
3,100997,Roger,Smith,R,BAH,185.0
4,101086,Ronald,Agenor,R,USA,180.0
...,...,...,...,...,...,...
2394,210013,Martin,Damm Sr,R,CZE,188.0
2395,210079,Jisung,Nam,U,KOR,NaN
2396,210107,Bor,Artnak,U,SLO,NaN
2397,210250,Erik,Arutiunian,U,BLR,NaN


#### Match data additional

In [10]:
whole_match_data_additional = whole_match_data_additional.drop(
    ["tourney_id", "tourney_name", "match_num","winner_seed", "loser_seed", "winner_rank_points", "loser_rank_points",
     "winner_entry", "loser_entry", "score", "minutes", "w_ace", "l_ace", "w_df", "l_df",
     "w_svpt", "l_svpt", "w_1stIn", "l_1stIn", "w_1stWon", "l_1stWon", "w_2ndWon", "l_2ndWon",
     "w_SvGms", "l_SvGms", "w_bpSaved", "l_bpSaved", "w_bpFaced", "l_bpFaced"], axis=1)

In [11]:
whole_match_data_additional.head(3)

,index,surface,draw_size,tourney_level,tourney_date,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,...,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,winner_rank,loser_rank
0,0,Hard,32,A,20000110,103163,Tommy Haas,R,188.0,GER,...,101543,Jeff Tarango,L,180.0,USA,31.137577,3,R32,11.0,63.0
1,1,Hard,32,A,20000110,102607,Juan Balcells,R,190.0,ESP,...,102644,Franco Squillari,L,183.0,ARG,24.386037,3,R32,211.0,49.0
2,2,Hard,32,A,20000110,103252,Alberto Martin,R,175.0,ESP,...,102238,Alberto Berasategui,R,173.0,ESP,26.535250,3,R32,48.0,59.0


### Clean nan values
#### (2000-2021).csv

In [12]:
# Get all nans
whole_match_data.isna().sum()

index           0
atp             0
location        0
date            0
series          0
court           0
surface         0
round           0
best_of         0
winner          0
loser           0
wrank          20
lrank         125
cbw         39628
cbl         39628
gbw         51646
gbl         51646
iww         43615
iwl         43615
sbw         51304
sbl         51304
b365w        8419
b365l        8399
b_ww        55482
b_wl        55482
exw         18621
exl         18616
psw         14460
psl         14460
ubw         46280
ubl         46280
lbw         29527
lbl         29516
sjw         41613
sjl         41606
maxw        28503
maxl        28503
avgw        28503
avgl        28503
dtype: int64

In [13]:
# Wrank and Lrank nans possibly can be dropped out since there is not much values and it is hard to restore it
whole_match_data = whole_match_data.drop(whole_match_data.index[whole_match_data["wrank"].isna() | whole_match_data["lrank"].isna()], axis=0)
# Remone "NR" outlier
whole_match_data = whole_match_data.drop(whole_match_data[whole_match_data["lrank"] == "NR"].index)

In [14]:
whole_match_data["lrank"] = whole_match_data["lrank"].astype(np.float64)

In [15]:
whole_match_data.head(3)

,index,atp,location,date,series,court,surface,round,best_of,winner,...,ubw,ubl,lbw,lbl,sjw,sjl,maxw,maxl,avgw,avgl
0,0,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Get columns names for all odds
cols_odds = pd.Index(['cbw', 'cbl', 'gbw', 'gbl', 'iww', 'iwl', 'sbw', 'sbl',
       'b365w', 'b365l', 'b_ww', 'b_wl', 'exw', 'exl', 'psw', 'psl', 'ubw',
       'ubl', 'lbw', 'lbl', 'sjw', 'sjl'])
cols_odds_winners = cols_odds[::2]
cols_odds_losers = cols_odds[1::2]
cols_odds

Index(['cbw', 'cbl', 'gbw', 'gbl', 'iww', 'iwl', 'sbw', 'sbl', 'b365w',
       'b365l', 'b_ww', 'b_wl', 'exw', 'exl', 'psw', 'psl', 'ubw', 'ubl',
       'lbw', 'lbl', 'sjw', 'sjl'],
      dtype='object')

In [17]:
# Fill nan in maxw / maxl / avgw / avgl if have at least one bet

whole_match_data["avgw"] = whole_match_data["avgw"].fillna(whole_match_data[cols_odds_winners].mean(axis=1))
whole_match_data["avgl"] = whole_match_data["avgl"].fillna(whole_match_data[cols_odds_losers].mean(axis=1))
whole_match_data["maxw"] = whole_match_data["maxw"].fillna(whole_match_data[cols_odds_winners].max(axis=1))
whole_match_data["maxl"] = whole_match_data["maxl"].fillna(whole_match_data[cols_odds_losers].max(axis=1))

In [18]:
# Now we can drop all odds except max and avg
whole_match_data = whole_match_data.drop(cols_odds, axis=1)

In [19]:
# Still have nans in some of odds, but it can be ignored in evaluation phase
whole_match_data.isna().sum()

index          0
atp            0
location       0
date           0
series         0
court          0
surface        0
round          0
best_of        0
winner         0
loser          0
wrank          0
lrank          0
maxw        3758
maxl        3756
avgw        3758
avgl        3756
dtype: int64

#### Player data additional

In [20]:
# Check nans
player_data_additional.isna().sum()

player_id        0
name_first       0
name_last        0
hand            25
ioc              0
height        1403
dtype: int64

In [21]:
# Hand nans is low amount, so can replace just with right hand
player_data_additional["hand"] = player_data_additional["hand"].fillna("R")

In [22]:
# Check nans
player_data_additional.isna().sum()

player_id        0
name_first       0
name_last        0
hand             0
ioc              0
height        1403
dtype: int64

In [23]:
# Get height data for countries
height_data = player_data_additional[["ioc", "height"]]
# Group by country
height_data = height_data.groupby("ioc").mean().reset_index()
# Get mean for all countries
height_data_all = height_data["height"].mean()
# Fill nan if countries don't have height records
height_data = height_data.fillna(height_data_all)
# Convert to int
height_data["height"] = height_data["height"].astype(int).astype(np.float64)
height_data = height_data.set_index("ioc")

In [24]:
for idx in player_data_additional.index:
    if np.isnan(player_data_additional.iloc[idx]["height"]):
        height = height_data.loc[player_data_additional.iloc[idx]["ioc"]]["height"]
        player_data_additional.iloc[idx] = player_data_additional.iloc[idx].fillna(height)

In [25]:
player_data_additional.isna().sum()

player_id     0
name_first    0
name_last     0
hand          0
ioc           0
height        0
dtype: int64

In [26]:
player_data_additional.head(3)

,player_id,name_first,name_last,hand,ioc,height
0,100644,Alexander,Zverev,R,GER,198.0
1,100754,Rafael,Avalos Brenes,R,CRC,175.0
2,100943,Girts,Dzelde,R,LAT,173.0


#### Match data additional

In [27]:
whole_match_data_additional.isna().sum()

index               0
surface             0
draw_size           0
tourney_level       0
tourney_date        0
winner_id           0
winner_name         0
winner_hand         9
winner_ht        2636
winner_ioc          0
winner_age          3
loser_id            0
loser_name          0
loser_hand         41
loser_ht         5171
loser_ioc           0
loser_age           0
best_of             0
round               0
winner_rank       522
loser_rank       1358
dtype: int64

In [28]:
whole_match_data_additional = whole_match_data_additional.drop(whole_match_data_additional[whole_match_data_additional["winner_rank"].isna()].index)
whole_match_data_additional = whole_match_data_additional.drop(whole_match_data_additional[whole_match_data_additional["loser_rank"].isna()].index)

In [29]:
whole_match_data_additional

,index,surface,draw_size,tourney_level,tourney_date,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,...,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,winner_rank,loser_rank
0,0,Hard,32,A,20000110,103163,Tommy Haas,R,188.0,GER,...,101543,Jeff Tarango,L,180.0,USA,31.137577,3,R32,11.0,63.0
1,1,Hard,32,A,20000110,102607,Juan Balcells,R,190.0,ESP,...,102644,Franco Squillari,L,183.0,ARG,24.386037,3,R32,211.0,49.0
2,2,Hard,32,A,20000110,103252,Alberto Martin,R,175.0,ESP,...,102238,Alberto Berasategui,R,173.0,ESP,26.535250,3,R32,48.0,59.0
3,3,Hard,32,A,20000110,103507,Juan Carlos Ferrero,R,183.0,ESP,...,103819,Roger Federer,R,185.0,SUI,18.422998,3,R32,45.0,61.0
4,4,Hard,32,A,20000110,102103,Michael Sell,R,180.0,USA,...,102765,Nicolas Escude,R,185.0,FRA,23.770021,3,R32,167.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65888,2721,Hard,4,D,20210917,200416,August Holmgren,R,NaN,DEN,...,208937,Kasidit Samrej,R,NaN,THA,20.629706,3,RR,905.0,1136.0
65889,2722,Hard,4,D,20210917,200416,August Holmgren,R,NaN,DEN,...,106397,Wishaya Trongcharoenchaikul,R,NaN,THA,26.433949,3,RR,905.0,767.0
65890,2723,Hard,4,D,20210917,208937,Kasidit Samrej,R,NaN,THA,...,134087,Johannes Ingildsen,R,NaN,DEN,24.202601,3,RR,1136.0,1546.0
65891,2724,Hard,4,D,20210918,123795,Altug Celikbilek,U,NaN,TUR,...,207669,Robert Strombachs,U,NaN,GER,21.995893,3,RR,170.0,671.0


In [30]:
# Function to fill nan of height by mean of players in country
def fillna_height(player_id, height, players_df=player_data_additional):
    if np.isnan(height):
        p_height = players_df[players_df["player_id"] == player_id].iloc[0]["height"]
        return p_height
    return height

In [31]:
# Fillna of heights
whole_match_data_additional["winner_ht"] = whole_match_data_additional[["winner_id", "winner_ht"]].apply(lambda x: fillna_height(x["winner_id"].astype(int), x["winner_ht"]), axis=1)
whole_match_data_additional["loser_ht"] = whole_match_data_additional[["loser_id", "loser_ht"]].apply(lambda x: fillna_height(x["loser_id"].astype(int), x["loser_ht"]), axis=1)

In [32]:
# Fillna of hand with right hand
whole_match_data_additional["winner_hand"] = whole_match_data_additional["winner_hand"].fillna("R")
whole_match_data_additional["loser_hand"] = whole_match_data_additional["loser_hand"].fillna("R")

In [33]:
whole_match_data_additional.isna().sum()

index            0
surface          0
draw_size        0
tourney_level    0
tourney_date     0
winner_id        0
winner_name      0
winner_hand      0
winner_ht        0
winner_ioc       0
winner_age       0
loser_id         0
loser_name       0
loser_hand       0
loser_ht         0
loser_ioc        0
loser_age        0
best_of          0
round            0
winner_rank      0
loser_rank       0
dtype: int64

## Integrate data

In [34]:
# First we need is to merge whole_match_data and whole_match_data_additional
# To get player info in match

In [35]:
whole_match_data.head(3)

,index,atp,location,date,series,court,surface,round,best_of,winner,loser,wrank,lrank,maxw,maxl,avgw,avgl
0,0,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63.0,77.0,NaN,NaN,NaN,NaN
1,1,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,Clement A.,5.0,56.0,NaN,NaN,NaN,NaN
2,2,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,40.0,655.0,NaN,NaN,NaN,NaN


In [36]:
whole_match_data_additional.head(3)

,index,surface,draw_size,tourney_level,tourney_date,winner_id,winner_name,winner_hand,winner_ht,winner_ioc,...,loser_id,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,best_of,round,winner_rank,loser_rank
0,0,Hard,32,A,20000110,103163,Tommy Haas,R,188.0,GER,...,101543,Jeff Tarango,L,180.0,USA,31.137577,3,R32,11.0,63.0
1,1,Hard,32,A,20000110,102607,Juan Balcells,R,190.0,ESP,...,102644,Franco Squillari,L,183.0,ARG,24.386037,3,R32,211.0,49.0
2,2,Hard,32,A,20000110,103252,Alberto Martin,R,175.0,ESP,...,102238,Alberto Berasategui,R,173.0,ESP,26.535250,3,R32,48.0,59.0


In [37]:
'''
Here are possible columns for join:
    (winner/loser)_name, surface, round, year (since there are different date representations), wrank, lrank
'''

# Check names representations in both dataframes and change to same format
whole_match_data[["winner", "loser"]].head(5)

,winner,loser
0,Dosedel S.,Ljubicic I.
1,Enqvist T.,Clement A.
2,Escude N.,Baccanello P.
3,Federer R.,Knippschild J.
4,Fromberg R.,Woodbridge T.


In [38]:
whole_match_data_additional[["winner_name", "loser_name"]].head(5)

,winner_name,loser_name
0,Tommy Haas,Jeff Tarango
1,Juan Balcells,Franco Squillari
2,Alberto Martin,Alberto Berasategui
3,Juan Carlos Ferrero,Roger Federer
4,Michael Sell,Nicolas Escude


In [39]:
# In main data we have format Last_name + First_initial
# In additional data we have format First_name + Last_name
# It is needed to change format in additional data to same as in main

def change_name_format(player_id, players_df=player_data_additional):
    player_info = players_df[players_df["player_id"] == player_id].iloc[0]
    
    last_name = player_info["name_last"]
    first_name = player_info["name_first"]
    
    return last_name + " " + first_name[0] + "."

#whole_match_data_additional["winner_name"]

whole_match_data_additional["winner_name"] = whole_match_data_additional["winner_id"].astype(int).apply(change_name_format)
whole_match_data_additional["loser_name"] = whole_match_data_additional["loser_id"].astype(int).apply(change_name_format)

In [40]:
whole_match_data_additional["tourney_date"] = pd.to_datetime(whole_match_data_additional["tourney_date"], format='%Y%m%d')

In [41]:
import datetime

# Check that day is monaday(This format in additional dataset)
def check_monday(match_date):
    if match_date.weekday() == 0:
        return True
    return False

# Change date to nearest previous monday
def change_to_nearest_monday(match_date):
    if match_date.weekday() == 0:
        return match_date
    return match_date + datetime.timedelta(days=-match_date.weekday())

whole_match_data_additional["tourney_date"].apply(check_monday).sum()

52443

In [42]:
whole_match_data_additional["tourney_date"].apply(change_to_nearest_monday).apply(check_monday).sum()

64275

In [43]:
whole_match_data_additional["tourney_date"] = whole_match_data_additional["tourney_date"].apply(change_to_nearest_monday)

In [44]:
# Convert date to datetime
whole_match_data["date"] = pd.to_datetime(whole_match_data["date"], format='%Y-%m-%d')

In [45]:
whole_match_data.head(3)

,index,atp,location,date,series,court,surface,round,best_of,winner,loser,wrank,lrank,maxw,maxl,avgw,avgl
0,0,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.,Ljubicic I.,63.0,77.0,NaN,NaN,NaN,NaN
1,1,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.,Clement A.,5.0,56.0,NaN,NaN,NaN,NaN
2,2,1,Adelaide,2000-01-03,International,Outdoor,Hard,1st Round,3,Escude N.,Baccanello P.,40.0,655.0,NaN,NaN,NaN,NaN


In [46]:
# Change all dates to first date of tournament
def to_first_day(atp, atp_date_df):
    return atp_date_df[atp_date_df["atp"] == atp].iloc[0]["date"]

# Iterate over all years
for year in range(2000, 2022):
    # Get mask for year
    mask = (whole_match_data["date"] >= datetime.datetime(year=year, month=1, day=1)) & (whole_match_data["date"] < datetime.datetime(year=year+1, month=1, day=1))
    # Get date of first match
    atp_date = whole_match_data[mask][["atp", "date"]].groupby("atp").min().reset_index()
    # Set date to first date
    whole_match_data["date"][mask] = whole_match_data[mask]["atp"].apply(lambda x: to_first_day(x, atp_date))

In [47]:
whole_match_data = whole_match_data.drop(["index", "atp"], axis=1)

In [48]:
whole_match_data["date"] = whole_match_data["date"].apply(change_to_nearest_monday)

In [49]:
len(set(whole_match_data["date"].unique()))

717

In [50]:
len(set(whole_match_data["date"].unique()) & set(whole_match_data_additional["tourney_date"].unique()))

702

In [51]:
# Format round values to same format in both dataframes
whole_match_data["round"].unique()

array(['1st Round', '2nd Round', 'Quarterfinals', 'Semifinals',
       'The Final', '3rd Round', '4th Round', 'Round Robin'], dtype=object)

In [52]:
# Format round values to same format in both dataframes
whole_match_data_additional["round"].unique()

array(['R32', 'R16', 'QF', 'SF', 'F', 'R64', 'R128', 'RR', 'BR', 'ER'],
      dtype=object)

In [53]:
round_data = whole_match_data[["location", "date", "round"]]
round_data["count"] = 1
round_data = round_data.groupby(["location", "date", "round"]).sum().reset_index()

def map_round(location, date, round_val, round_data=round_data):
    
    def next_power_of_2(x):  
        return 1 if x == 0 else 2**(x - 1).bit_length()
    
    n_matches = round_data[(round_data["location"] == location) & (round_data["date"] == date) & (round_data["round"] == round_val)].iloc[0]["count"]
    n_matches = next_power_of_2(int(n_matches))
    
    mapping = {1: 'F', 2: 'SF', 4: 'QF', 8: 'R16', 16: 'R32', 32: 'R64', 64: 'R128'}
    
    return 'RR' if round_val == 'Round Robin' else mapping[n_matches]

whole_match_data['round'] = whole_match_data[["location", "date", "round"]].apply(lambda x: map_round(x["location"], x["date"], x["round"]), axis=1)

In [54]:
whole_match_data

,location,date,series,court,surface,round,best_of,winner,loser,wrank,lrank,maxw,maxl,avgw,avgl
0,Adelaide,2000-01-03,International,Outdoor,Hard,R32,3,Dosedel S.,Ljubicic I.,63.0,77.0,NaN,NaN,NaN,NaN
1,Adelaide,2000-01-03,International,Outdoor,Hard,R32,3,Enqvist T.,Clement A.,5.0,56.0,NaN,NaN,NaN,NaN
2,Adelaide,2000-01-03,International,Outdoor,Hard,R32,3,Escude N.,Baccanello P.,40.0,655.0,NaN,NaN,NaN,NaN
3,Adelaide,2000-01-03,International,Outdoor,Hard,R32,3,Federer R.,Knippschild J.,65.0,87.0,NaN,NaN,NaN,NaN
4,Adelaide,2000-01-03,International,Outdoor,Hard,R32,3,Fromberg R.,Woodbridge T.,81.0,198.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58659,Turin,2021-11-08,Masters Cup,Indoor,Hard,RR,3,Ruud C.,Rublev A.,8.0,5.0,2.65,1.63,2.49,1.53
58660,Turin,2021-11-08,Masters Cup,Indoor,Hard,RR,3,Djokovic N.,Norrie C.,1.0,12.0,1.16,7.71,1.12,6.27
58661,Turin,2021-11-08,Masters Cup,Indoor,Hard,SF,3,Medvedev D.,Ruud C.,2.0,8.0,1.18,7.40,1.13,5.83
58662,Turin,2021-11-08,Masters Cup,Indoor,Hard,SF,3,Zverev A.,Djokovic N.,3.0,1.0,3.32,1.45,2.99,1.39


In [120]:
merged = whole_match_data.merge(whole_match_data_additional, left_on=["wrank", "lrank", "surface", "best_of", "date", "round"], right_on=["winner_rank", "loser_rank", "surface", "best_of", "tourney_date", "round"])

In [121]:
merged.columns

Index(['location', 'date', 'series', 'court', 'surface', 'round', 'best_of',
       'winner', 'loser', 'wrank', 'lrank', 'maxw', 'maxl', 'avgw', 'avgl',
       'index', 'draw_size', 'tourney_level', 'tourney_date', 'winner_id',
       'winner_name', 'winner_hand', 'winner_ht', 'winner_ioc', 'winner_age',
       'loser_id', 'loser_name', 'loser_hand', 'loser_ht', 'loser_ioc',
       'loser_age', 'winner_rank', 'loser_rank'],
      dtype='object')

In [122]:
# Winner and loser same as winner_name and loser_name
# Index not needed
# Winner_id, loser_id not needed
# series and tourney_level are the same
# Draw_size info have in round
# wrank and lrank same as winner_rank and loser_ranl
merged = merged.drop(["winner", "loser", "draw_size", "index", "winner_id", "loser_id", "series", "tourney_date", "wrank", "lrank"], axis=1)
merged

,location,date,court,surface,round,best_of,maxw,maxl,avgw,avgl,...,winner_ht,winner_ioc,winner_age,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,winner_rank,loser_rank
0,Adelaide,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,183.0,CZE,29.368925,Ljubicic I.,R,193.0,CRO,20.774812,63.0,77.0
1,Adelaide,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,190.0,USA,22.565366,Arthurs W.,L,190.0,AUS,28.777550,58.0,105.0
2,Doha,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,196.0,BLR,22.475017,Puerta M.,L,180.0,ARG,21.270363,74.0,101.0
3,Doha,2000-01-03,Outdoor,Hard,QF,3,NaN,NaN,NaN,NaN,...,183.0,GER,22.477755,Schalken S.,R,193.0,NED,23.299110,6.0,44.0
4,Auckland,2000-01-10,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,190.0,ESP,24.558522,Squillari F.,L,183.0,ARG,24.386037,211.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47148,Paris,2021-11-01,Indoor,Hard,QF,3,1.15,10.10,1.09,7.35,...,198.0,RUS,25.722108,Gaston H.,L,184.0,FRA,21.097878,2.0,103.0
47149,Paris,2021-11-01,Indoor,Hard,QF,3,1.43,3.36,1.37,3.06,...,198.0,GER,24.533881,Ruud C.,R,183.0,NOR,22.861054,4.0,8.0
47150,Paris,2021-11-01,Indoor,Hard,SF,3,1.20,6.30,1.16,5.21,...,188.0,SRB,34.447639,Hurkacz H.,R,196.0,POL,24.720055,1.0,10.0
47151,Paris,2021-11-01,Indoor,Hard,SF,3,1.92,2.18,1.79,2.03,...,198.0,RUS,25.722108,Zverev A.,R,198.0,GER,24.533881,2.0,4.0


In [123]:
merged.isna().sum()

location            0
date                0
court               0
surface             0
round               0
best_of             0
maxw             3401
maxl             3399
avgw             3401
avgl             3399
tourney_level       0
winner_name         0
winner_hand         0
winner_ht           0
winner_ioc          0
winner_age          0
loser_name          0
loser_hand          0
loser_ht            0
loser_ioc           0
loser_age           0
winner_rank         0
loser_rank          0
dtype: int64

In [124]:
# Check 
merged[["winner_name", "loser_name"]].head(10)

,winner_name,loser_name
0,Dosedel S.,Ljubicic I.
1,Gambill J.,Arthurs W.
2,Mirnyi M.,Puerta M.
3,Kiefer N.,Schalken S.
4,Balcells J.,Squillari F.
5,Behrend T.,Hantschk M.
6,Chang M.,Black B.
7,Ferrero J.,Federer R.
8,Gambill J.,Fromberg R.
9,Gaudio G.,Sargsian S.


In [125]:
# Permutate winner and loser columns to prevent correlation between target value

winner_cols = ["winner_name", "winner_hand", "winner_ht", "winner_ioc", "winner_age", "winner_rank"]
loser_cols = ["loser_name", "loser_hand", "loser_ht", "loser_ioc", "loser_age", "loser_rank"]
permutate_cols = winner_cols + loser_cols + ["player_1_won", "player_2_won"]

# Function to permutate
def permutate(x):
    winner_info = list(x[winner_cols])
    loser_info = list(x[loser_cols])
    
    rand = np.random.uniform(0, 1)
    
    if rand < 0.5:
        return pd.Series(winner_info + loser_info + [1, 0])
    else:
        return pd.Series(loser_info + winner_info + [0, 1])

merged[permutate_cols] = merged[winner_cols+loser_cols].apply(lambda x: permutate(x), axis=1)

rename_dict = {}
for winner_label in winner_cols:
    rename_dict[winner_label] = winner_label.replace('winner', 'player_1')
    
for loser_label in loser_cols:
    rename_dict[loser_label] = loser_label.replace('loser', 'player_2')
    
merged = merged.rename(columns=rename_dict)

In [126]:
# Check permutation
merged[["player_1_name", "player_2_name"]].head(10)

,player_1_name,player_2_name
0,Ljubicic I.,Dosedel S.
1,Arthurs W.,Gambill J.
2,Mirnyi M.,Puerta M.
3,Kiefer N.,Schalken S.
4,Balcells J.,Squillari F.
5,Hantschk M.,Behrend T.
6,Chang M.,Black B.
7,Ferrero J.,Federer R.
8,Gambill J.,Fromberg R.
9,Sargsian S.,Gaudio G.


## Format data

### Encoding format

| # | attributes | encode | reasons |
|:---:|:---|:---|:---|
| 1 | location | label encoding | A lot of values. Encoding with one-hot is not preferrable, because it is possible to add new location further, but in one-hot we need one more attribute for this |
| 2 | date | Split to year-month-day and encode with Cyclical | Some players may play better in different seasons. We need to maintain this info. |
| 3 | court | One-hot | No new values are added. |
| 4 | surface | One-hot | No new values are added.No |
| 5 | round | One-hot | No new values are added. |
| 6 | best-of | One-hot | No new values are added. |
| 7 | player_(1/2)_rank | StandardScale | Just standart scale for float data |
| 8 | maxw/maxl/avgw/avgl | No Scale | Used for evaluation only |
| 9 | tourney_level | One-hot/Label | No new values are added / have info about order (A>B>C...) |
| 10 | player_(1/2)_name | Label encoding | A lot of new values may be added |
| 11 | player_(1/2)_hand | One-hot | No new values are added. |
| 12 | player_(1/2)_ht | StandardScale | In height we assume to see a normal distribution |
| 13 | player_(1/2)_ioc | One-hot | No new values are added. |
| 14 | player_(1/2)_age | StandardScale | In age we assume to see a normal distribution |

In [127]:
# Save raw info
merged.to_csv("Data/final_data_raw.csv")

In [128]:
# Label encoding
# Location, winner_name/loser_name encode

merged["location"] = merged["location"].astype('category').cat.codes

conc_names = pd.concat([merged["player_1_name"], merged["player_2_name"]]).astype('category').cat.codes
merged["player_1_name"] = conc_names.iloc[:len(conc_names)//2]
merged["player_2_name"] = conc_names.iloc[len(conc_names)//2:]
merged.head(3)

,location,date,court,surface,round,best_of,maxw,maxl,avgw,avgl,...,player_1_age,player_2_name,player_2_hand,player_2_ht,player_2_ioc,player_2_age,player_1_rank,player_2_rank,player_1_won,player_2_won
0,2,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,20.774812,290,R,183.0,CZE,29.368925,77.0,63.0,0,1
1,2,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,28.777550,382,R,190.0,USA,22.565366,105.0,58.0,0,1
2,30,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,22.475017,939,L,180.0,ARG,21.270363,74.0,101.0,1,0


In [129]:
from feature_engine.creation import CyclicalTransformer

# Date encode
# Don't drop date. It is needed to train-test split
merged["year"] = merged["date"].apply(lambda x: x.year)
merged["month"] = merged["date"].apply(lambda x: x.month)
merged["day"] = merged["date"].apply(lambda x: x.day)

cyclical = CyclicalTransformer(variables=None, drop_original=True)
merged = pd.concat([merged, cyclical.fit_transform(merged[["year", "month", "day"]])], axis=1)
merged = merged.drop(["year", "month", "day"], axis=1)
merged.head(3)

,location,date,court,surface,round,best_of,maxw,maxl,avgw,avgl,...,player_1_rank,player_2_rank,player_1_won,player_2_won,year_sin,year_cos,month_sin,month_cos,day_sin,day_cos
0,2,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,77.0,63.0,0,1,-0.065242,0.99787,0.5,0.866025,0.571268,0.820763
1,2,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,105.0,58.0,0,1,-0.065242,0.99787,0.5,0.866025,0.571268,0.820763
2,30,2000-01-03,Outdoor,Hard,R32,3,NaN,NaN,NaN,NaN,...,74.0,101.0,1,0,-0.065242,0.99787,0.5,0.866025,0.571268,0.820763


In [130]:
# One-hot encoding
# Court, surface, round, best_of, tourney_level, winner_hand/loser_hand, winner_ioc/loser_ioc encoding

merged = pd.concat([merged, pd.get_dummies(merged["court"])], axis=1).drop("court", axis=1)
merged = pd.concat([merged, pd.get_dummies(merged["surface"])], axis=1).drop("surface", axis=1)
merged = pd.concat([merged, pd.get_dummies(merged["round"])], axis=1).drop("round", axis=1)
merged = pd.concat([merged, pd.get_dummies(merged["best_of"])], axis=1).drop("best_of", axis=1)
merged['tourney_level'] = merged['tourney_level'].apply(lambda x: x + '_level')
merged = pd.concat([merged, pd.get_dummies(merged["tourney_level"])], axis=1).drop("tourney_level", axis=1)

ohe_conc_players_hand = pd.get_dummies(pd.concat([merged["player_1_hand"], merged["player_2_hand"]]))
ohe_conc_players_hand = ohe_conc_players_hand.rename(lambda x: x + '_p1', axis='columns')
merged = pd.concat([merged, ohe_conc_players_hand.iloc[:len(ohe_conc_players_hand)//2]], axis=1).drop("player_1_hand", axis=1)
ohe_conc_players_hand = ohe_conc_players_hand.rename(lambda x: x[:-3] + '_p2', axis='columns')
merged = pd.concat([merged, ohe_conc_players_hand.iloc[len(ohe_conc_players_hand)//2:]], axis=1).drop("player_2_hand", axis=1)

ohe_conc_players_ioc = pd.get_dummies(pd.concat([merged["player_1_ioc"], merged["player_2_ioc"]]))
ohe_conc_players_ioc = ohe_conc_players_ioc.rename(lambda x: x + '_p1', axis='columns')
merged = pd.concat([merged, ohe_conc_players_ioc.iloc[:len(ohe_conc_players_ioc)//2]], axis=1).drop("player_1_ioc", axis=1)
ohe_conc_players_ioc = ohe_conc_players_ioc.rename(lambda x: x[:-3] + '_p2', axis='columns')
merged = pd.concat([merged, ohe_conc_players_ioc.iloc[len(ohe_conc_players_ioc)//2:]], axis=1).drop("player_2_ioc", axis=1)
merged.head(3)

,location,date,maxw,maxl,avgw,avgl,player_1_name,player_1_ht,player_1_age,player_2_name,...,TPE_p2,TUN_p2,TUR_p2,UAE_p2,UKR_p2,URU_p2,USA_p2,UZB_p2,VEN_p2,ZIM_p2
0,2,2000-01-03,NaN,NaN,NaN,NaN,679,193.0,20.774812,290,...,0,0,0,0,0,0,0,0,0,0
1,2,2000-01-03,NaN,NaN,NaN,NaN,45,190.0,28.777550,382,...,0,0,0,0,0,0,1,0,0,0
2,30,2000-01-03,NaN,NaN,NaN,NaN,784,196.0,22.475017,939,...,0,0,0,0,0,0,0,0,0,0


In [131]:
def train_test_split(df, year_split): # X_train, y_train, X_test, y_test
    df = df.sort_values(by=['date'])
    
    mask = df['date'] <= str(year_split)
    
    X_train = df[mask].drop(['player_1_won', 'player_2_won', 'date', 'maxw', 'maxl', 'avgw', 'avgl'], axis=1)
    y_train = df[mask][['player_1_won', 'player_2_won', 'date', 'maxw', 'maxl', 'avgw', 'avgl']]
    
    X_test = df[~mask].drop(['player_1_won', 'player_2_won', 'date', 'maxw', 'maxl', 'avgw', 'avgl'], axis=1)
    y_test = df[~mask][['player_1_won', 'player_2_won', 'date', 'maxw', 'maxl', 'avgw', 'avgl']]

    return X_train, y_train, X_test, y_test

In [132]:
X_train, y_train, X_test, y_test = train_test_split(merged, 2019)

In [133]:
# Standard Scaling
# wrank/lrank, winner_ht/loser_ht, winner_age/loser_age scale


def std_scale(series, mean, std):
    return (series-mean)/std

concated = pd.concat([X_train["player_1_rank"], X_train["player_2_rank"]])
mean = concated.mean()
std = concated.std()
X_train["player_1_rank"] = std_scale(X_train["player_1_rank"], mean, std)
X_train["player_2_rank"] = std_scale(X_train["player_2_rank"], mean, std)

X_test["player_1_rank"] = std_scale(X_test["player_1_rank"], mean, std)
X_test["player_2_rank"] = std_scale(X_test["player_2_rank"], mean, std)


concated = pd.concat([X_train["player_1_ht"], X_train["player_2_ht"]])
mean = concated.mean()
std = concated.std()
X_train["player_1_ht"] = std_scale(X_train["player_1_ht"], mean, std)
X_train["player_2_ht"] = std_scale(X_train["player_2_ht"], mean, std)

X_test["player_1_ht"] = std_scale(X_test["player_1_ht"], mean, std)
X_test["player_2_ht"] = std_scale(X_test["player_2_ht"], mean, std)


concated = pd.concat([X_train["player_1_age"], X_train["player_2_age"]])
mean = concated.mean()
std = concated.std()
X_train["player_1_age"] = std_scale(X_train["player_1_age"], mean, std)
X_train["player_2_age"] = std_scale(X_train["player_2_age"], mean, std)

X_test["player_1_age"] = std_scale(X_test["player_1_age"], mean, std)
X_test["player_2_age"] = std_scale(X_test["player_2_age"], mean, std)

In [134]:
# Check columns
list(X_train.columns)

['location',
 'player_1_name',
 'player_1_ht',
 'player_1_age',
 'player_2_name',
 'player_2_ht',
 'player_2_age',
 'player_1_rank',
 'player_2_rank',
 'year_sin',
 'year_cos',
 'month_sin',
 'month_cos',
 'day_sin',
 'day_cos',
 'Indoor',
 'Outdoor',
 'Carpet',
 'Clay',
 'Grass',
 'Hard',
 'F',
 'QF',
 'R128',
 'R16',
 'R32',
 'R64',
 'RR',
 'SF',
 3,
 5,
 'A_level',
 'F_level',
 'G_level',
 'M_level',
 'L_p1',
 'R_p1',
 'U_p1',
 'L_p2',
 'R_p2',
 'U_p2',
 'ALG_p1',
 'ARG_p1',
 'ARM_p1',
 'AUS_p1',
 'AUT_p1',
 'AZE_p1',
 'BAH_p1',
 'BAR_p1',
 'BEL_p1',
 'BIH_p1',
 'BLR_p1',
 'BOL_p1',
 'BRA_p1',
 'BUL_p1',
 'CAN_p1',
 'CHI_p1',
 'CHN_p1',
 'COL_p1',
 'CRC_p1',
 'CRO_p1',
 'CYP_p1',
 'CZE_p1',
 'DEN_p1',
 'DOM_p1',
 'ECU_p1',
 'EGY_p1',
 'ESA_p1',
 'ESP_p1',
 'EST_p1',
 'FIN_p1',
 'FRA_p1',
 'GBR_p1',
 'GEO_p1',
 'GER_p1',
 'GRE_p1',
 'HKG_p1',
 'HUN_p1',
 'IND_p1',
 'IRL_p1',
 'ISR_p1',
 'ITA_p1',
 'JPN_p1',
 'KAZ_p1',
 'KOR_p1',
 'KUW_p1',
 'LAT_p1',
 'LTU_p1',
 'LUX_p1',
 'MAR_p1',


In [135]:
# Save encoded info
X_train.to_csv("Data/X_train.csv")
X_test.to_csv("Data/X_test.csv")
y_train.to_csv("Data/y_train.csv")
y_test.to_csv("Data/y_test.csv")